In [4]:
import torch
from preprocessing import get_dataframes
from dataloading import Text_dataset, get_dataloaders
from utilities import set_seed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

# Enforce reproducability
set_seed(42)

# Create dataframes
train_url = "https://raw.githubusercontent.com/brendenlake/SCAN/master/length_split/tasks_train_length.txt"
test_url = "https://raw.githubusercontent.com/brendenlake/SCAN/master/length_split/tasks_test_length.txt"
train_df, test_df, voc_in, voc_out = get_dataframes(train_url, test_url)

# Create datasets
train_data = Text_dataset(train_df[['IN_idx', 'OUT_idx', 'lens']], sample=True, size=10000)
test_data = Text_dataset(test_df[['IN_idx', 'OUT_idx', 'lens']], sample=False)

# Create
train_loader, test_loader = get_dataloaders(train_data, test_data)

Device:  cpu
Random seed set as 42


In [ ]:
import models
from training import train

# Best in experiment 2
rnn = 'gru'
layers = 1
hidden_size = 50
dropout = 0.5
max_len = max(test_df.lens)

for i in range(1,5):
    set_seed(i)
    encoder = models.EncoderRNN(rnn, len(voc_in), hidden_size, layers, dropout).to(device)
    decoder = models.AttnDecoderRNN(rnn, hidden_size, len(voc_out), layers, device, max_len, dropout).to(device)      
    train(train_loader, encoder, decoder, device, len(voc_out), save_name='ex2_best_'+str(i), )

In [ ]:
import models
from training import evaluate

# Best in experiment 2
rnn = 'gru'
layers = 1
hidden_size = 50
dropout = 0.5
max_len = max(test_df.lens)

acc_oracle = 0
acc_normal = 0
for i in range(5):
    set_seed(i)
    encoder = models.EncoderRNN(rnn, len(voc_in), hidden_size, layers, dropout).to(device)
    decoder = models.AttnDecoderRNN(rnn, hidden_size, len(voc_out), layers, device, max_len, dropout).to(device)
    encoder.load_state_dict(torch.load('models/encoder_ex2_best_'+str(i)+'.pth'))
    decoder.load_state_dict(torch.load('models/decoder_ex2_best_'+str(i)+'.pth'))
    acc_oracle += evaluate(encoder, decoder, test_loader, device, oracle=True)
    acc_normal += evaluate(encoder, decoder, test_loader, device, oracle=False)

txt = "GRU Normal: {}\nGRU w. oracle: {}".format(acc_normal/5, acc_oracle/5)
f = open("ex2_oracle_gru", "a")
f.write(txt)
f.close()

In [ ]:
import models
from training import train

# Overall best
rnn = 'lstm'
layers = 2
hidden_size = 200
dropout = 0.5
max_len = max(test_df.lens)

for i in range(5):
    set_seed(i)
    encoder = models.EncoderRNN(rnn, len(voc_in), hidden_size, layers, dropout).to(device)
    decoder = models.DecoderRNN(rnn, hidden_size, len(voc_out), layers, device, max_len).to(device)      
    train(train_loader, encoder, decoder, device, len(voc_out), save_name='ex2_overall_'+str(i))

In [ ]:
acc_oracle += evaluate(encoder, decoder, test_loader, device, oracle=True)
acc_normal += evaluate(encoder, decoder, test_loader, device, oracle=False)

In [5]:
import models
from training import evaluate

# Overall best
rnn = 'lstm'
layers = 2
hidden_size = 200
dropout = 0.5
max_len = max(test_df.lens)

acc_oracle = 0
acc_normal = 0
for i in range(1):
    set_seed(i)
    encoder = models.EncoderRNN(rnn, len(voc_in), hidden_size, layers, dropout).to(device)
    decoder = models.DecoderRNN(rnn, hidden_size, len(voc_out), layers, device, max_len).to(device)
    encoder.load_state_dict(torch.load('models/encoder_ex2_overall_'+str(i)+'.pth'))
    decoder.load_state_dict(torch.load('models/decoder_ex2_overall_'+str(i)+'.pth'))
    acc_oracle += evaluate(encoder, decoder, train_loader, device, oracle=True)
    acc_normal += evaluate(encoder, decoder, train_loader, device, oracle=False)

txt = "LSTM Normal: {}\nLSTM w. oracle: {}".format(acc_normal/5, acc_oracle/5)
f = open("ex2_oracle_LSTM", "a")
f.write(txt)
f.close()

Random seed set as 0


  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [01:29<00:00, 111.59it/s]


In [ ]:
torch.tensor([3,4,2]).topk(2)

In [ ]:
"""
import pytorch_lightning as pl
import models
pl.seed_everything(42)

# Best in experiment 2
rnn = 'gru'
layers = 1
hidden_size = 50
dropout = 0.5
max_len = train_df.OUT_idx.apply(len).max()

encoder = models.EncoderRNN(rnn, len(voc_in), hidden_size, layers, dropout)
decoder = models.AttnDecoderRNN(rnn, hidden_size, len(voc_out), layers, device, max_len, dropout)
seq2seq = models.Seq2SeqModel(encoder, decoder, len(train_loader))
trainer = pl.Trainer(gradient_clip_val=5, max_epochs=1, log_every_n_steps=1000)
print("Fitting model..")
trainer.fit(model=seq2seq, train_dataloaders=train_loader)
"""